In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
max_nodes = 150
dataset_sparse = PygGraphPropPredDataset(root='/data/XXX/Pooling', name="ogbg-molpcba")
evaluator = Evaluator("ogbg-molpcba")
eval_metric = dataset_sparse.eval_metric
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
dataset_sparse = dataset_sparse.shuffle()
num_total = len(dataset_sparse)
num_train = int(num_total * train_ratio)
num_val = int(num_total * val_ratio)
num_test = num_total - num_train - num_val
train_dataset = dataset_sparse[:num_train]
val_dataset = dataset_sparse[num_train:num_train + num_val]
test_dataset = dataset_sparse[num_train + num_val:]
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### TopKPooling with HierarchicalGCN (2019)

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_TOPK(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_TOPK, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = TopKPooling(hidden_channels, ratio=0.3)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = TopKPooling(hidden_channels, ratio=0.3)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, edge_attr, batch, _, _ = self.pool1(x, edge_index, edge_attr, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, edge_attr, batch, _, _ = self.pool2(x, edge_index, edge_attr, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_TOPK(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=4096, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False)
    model = HierarchicalGCN_TOPK(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0623, Val Acc: 0.1137, Test Acc: 0.1152
Seed: 42, Epoch: 002, Loss: 0.0605, Val Acc: 0.1212, Test Acc: 0.1235
Seed: 42, Epoch: 003, Loss: 0.0605, Val Acc: 0.1182, Test Acc: 0.1201
Seed: 42, Epoch: 004, Loss: 0.0603, Val Acc: 0.1206, Test Acc: 0.1223
Seed: 42, Epoch: 005, Loss: 0.0601, Val Acc: 0.1180, Test Acc: 0.1199
Seed: 42, Epoch: 006, Loss: 0.0600, Val Acc: 0.1234, Test Acc: 0.1262
Seed: 42, Epoch: 007, Loss: 0.0598, Val Acc: 0.1242, Test Acc: 0.1280
Seed: 42, Epoch: 008, Loss: 0.0597, Val Acc: 0.1233, Test Acc: 0.1254
Seed: 42, Epoch: 009, Loss: 0.0596, Val Acc: 0.1199, Test Acc: 0.1221
Seed: 42, Epoch: 010, Loss: 0.0595, Val Acc: 0.1224, Test Acc: 0.1248
Seed: 42, Epoch: 011, Loss: 0.0595, Val Acc: 0.1252, Test Acc: 0.1277
Seed: 42, Epoch: 012, Loss: 0.0591, Val Acc: 0.1321, Test Acc: 0.1349
Seed: 42, Epoch: 013, Loss: 0.0589, Val Acc: 0.1287, Test Acc: 0.1322
Seed: 42, Epoch: 014, Loss: 0.0588, Val Acc: 0.1336, Test Acc: 0.1377
Seed: 42, Epoch: 015

### SAGPooling with HierarchicalGCN (2019)

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_SAG(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_SAG, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = SAGPooling(hidden_channels, ratio=0.7)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = SAGPooling(hidden_channels, ratio=0.7)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, edge_attr, batch, _, _ = self.pool1(x, edge_index, edge_attr, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, edge_attr, batch, _, _ = self.pool2(x, edge_index, edge_attr, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_SAG(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=4096, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False)
    model = HierarchicalGCN_SAG(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0647, Val AP: 0.1107, Test AP: 0.1148
Seed: 42, Epoch: 002, Loss: 0.0610, Val AP: 0.1158, Test AP: 0.1187
Seed: 42, Epoch: 003, Loss: 0.0604, Val AP: 0.1163, Test AP: 0.1196
Seed: 42, Epoch: 004, Loss: 0.0603, Val AP: 0.1176, Test AP: 0.1204
Seed: 42, Epoch: 005, Loss: 0.0603, Val AP: 0.1188, Test AP: 0.1197
Seed: 42, Epoch: 006, Loss: 0.0601, Val AP: 0.1184, Test AP: 0.1203
Seed: 42, Epoch: 007, Loss: 0.0599, Val AP: 0.1195, Test AP: 0.1213
Seed: 42, Epoch: 008, Loss: 0.0596, Val AP: 0.1210, Test AP: 0.1232
Seed: 42, Epoch: 009, Loss: 0.0596, Val AP: 0.1219, Test AP: 0.1241
Seed: 42, Epoch: 010, Loss: 0.0597, Val AP: 0.1224, Test AP: 0.1249
Seed: 42, Epoch: 011, Loss: 0.0596, Val AP: 0.1220, Test AP: 0.1245
Seed: 42, Epoch: 012, Loss: 0.0595, Val AP: 0.1244, Test AP: 0.1270
Seed: 42, Epoch: 013, Loss: 0.0594, Val AP: 0.1218, Test AP: 0.1245
Seed: 42, Epoch: 014, Loss: 0.0593, Val AP: 0.1244, Test AP: 0.1277
Seed: 42, Epoch: 015, Loss: 0.0592, Val AP: 0.12

### ASAPooling with HierarchicalGCN (2019)

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_ASA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_ASA, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=4096, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False)
    model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

### PANPooling with HierarchicalGCN

In [2]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
class PANPooling(torch.nn.Module):
    r""" General Graph pooling layer based on PAN, which can work with all layers.
    """
    def __init__(self, in_channels, ratio=0.5, pan_pool_weight=None, min_score=None, multiplier=1,
                 nonlinearity=torch.tanh, filter_size=3, panpool_filter_weight=None):
        super(PANPooling, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.min_score = min_score
        self.multiplier = multiplier
        self.nonlinearity = nonlinearity
        self.filter_size = filter_size
        if panpool_filter_weight is None:
            self.panpool_filter_weight = torch.nn.Parameter(0.5 * torch.ones(filter_size), requires_grad=True)
        self.transform = Parameter(torch.ones(in_channels), requires_grad=True)
        if pan_pool_weight is None:
            self.pan_pool_weight = torch.nn.Parameter(0.5 * torch.ones(2), requires_grad=True)
        else:
            self.pan_pool_weight = pan_pool_weight
    def forward(self, x, edge_index, M=None, batch=None, num_nodes=None):
        """"""
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        edge_index, edge_weight = self.panentropy_sparse(edge_index, num_nodes)
        num_nodes = x.size(0)
        degree = torch.zeros(num_nodes, device=edge_index.device)
        degree = scatter_add(edge_weight, edge_index[0], out=degree)
        xtransform = torch.matmul(x, self.transform)
        x_transform_norm = xtransform 
        degree_norm = degree 
        score = self.pan_pool_weight[0] * x_transform_norm + self.pan_pool_weight[1] * degree_norm
        if self.min_score is None:
            score = self.nonlinearity(score)
        else:
            score = softmax(score, batch)
        perm = self.topk(score, self.ratio, batch, self.min_score)
        x = x[perm] * score[perm].view(-1, 1)
        x = self.multiplier * x if self.multiplier != 1 else x
        batch = batch[perm]
        edge_index, edge_weight = self.filter_adj(edge_index, edge_weight, perm, num_nodes=score.size(0))
        return x, edge_index, edge_weight, batch, perm, score[perm]
    def topk(self, x, ratio, batch, min_score=None, tol=1e-7):
        if min_score is not None:
            scores_max = scatter_max(x, batch)[0][batch] - tol
            scores_min = scores_max.clamp(max=min_score)
            perm = torch.nonzero(x > scores_min).view(-1)
        else:
            num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
            batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
            cum_num_nodes = torch.cat(
                [num_nodes.new_zeros(1),
                 num_nodes.cumsum(dim=0)[:-1]], dim=0)
            index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
            index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
            dense_x = x.new_full((batch_size * max_num_nodes, ), -2)
            dense_x[index] = x
            dense_x = dense_x.view(batch_size, max_num_nodes)
            _, perm = dense_x.sort(dim=-1, descending=True)
            perm = perm + cum_num_nodes.view(-1, 1)
            perm = perm.view(-1)
            k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
            mask = [
                torch.arange(k[i], dtype=torch.long, device=x.device) +
                i * max_num_nodes for i in range(batch_size)
            ]
            mask = torch.cat(mask, dim=0)
            perm = perm[mask]
        return perm
    def filter_adj(self, edge_index, edge_weight, perm, num_nodes=None):
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        mask = perm.new_full((num_nodes, ), -1)
        i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
        mask[perm] = i
        row, col = edge_index
        row, col = mask[row], mask[col]
        mask = (row >= 0) & (col >= 0)
        row, col = row[mask], col[mask]
        if edge_weight is not None:
            edge_weight = edge_weight[mask]
        return torch.stack([row, col], dim=0), edge_weight
    def panentropy_sparse(self, edge_index, num_nodes):
        edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
        edge_index, edge_value = coalesce(edge_index, edge_value, num_nodes, num_nodes)
        pan_index, pan_value = eye(num_nodes, device=edge_index.device)
        indextmp = pan_index.clone().to(edge_index.device)
        valuetmp = pan_value.clone().to(edge_index.device)
        pan_value = self.panpool_filter_weight[0] * pan_value
        for i in range(self.filter_size - 1):
            indextmp, valuetmp = spspmm(indextmp, valuetmp, edge_index, edge_value, num_nodes, num_nodes, num_nodes)
            valuetmp = valuetmp * self.panpool_filter_weight[i+1]
            indextmp, valuetmp = coalesce(indextmp, valuetmp, num_nodes, num_nodes)
            pan_index = torch.cat((pan_index, indextmp), 1)
            pan_value = torch.cat((pan_value, valuetmp))
        return coalesce(pan_index, pan_value, num_nodes, num_nodes, op='add')

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_PAN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_PAN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = PANPooling(hidden_channels, ratio=0.7, pan_pool_weight=None, min_score=None, multiplier=1,
                 nonlinearity=torch.tanh, filter_size=1, panpool_filter_weight=None)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = PANPooling(hidden_channels, ratio=0.7, pan_pool_weight=None, min_score=None, multiplier=1,
                 nonlinearity=torch.tanh, filter_size=1, panpool_filter_weight=None)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, perm, score_perm = self.pool1(x, edge_index, batch=batch, M=None)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, perm, score_perm = self.pool2(x, edge_index, batch=batch, M=None)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_PAN(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=4096, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False)
    model = HierarchicalGCN_PAN(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0641, Val AP: 0.1125, Test AP: 0.1075
Seed: 42, Epoch: 002, Loss: 0.0608, Val AP: 0.1230, Test AP: 0.1193
Seed: 42, Epoch: 003, Loss: 0.0604, Val AP: 0.1211, Test AP: 0.1178
Seed: 42, Epoch: 004, Loss: 0.0603, Val AP: 0.1168, Test AP: 0.1154
Seed: 42, Epoch: 005, Loss: 0.0602, Val AP: 0.1153, Test AP: 0.1143
Seed: 42, Epoch: 006, Loss: 0.0599, Val AP: 0.1193, Test AP: 0.1181
Seed: 42, Epoch: 007, Loss: 0.0596, Val AP: 0.1222, Test AP: 0.1203
Seed: 42, Epoch: 008, Loss: 0.0596, Val AP: 0.1247, Test AP: 0.1218
Seed: 42, Epoch: 009, Loss: 0.0594, Val AP: 0.1189, Test AP: 0.1175
Seed: 42, Epoch: 010, Loss: 0.0592, Val AP: 0.1281, Test AP: 0.1254
Seed: 42, Epoch: 011, Loss: 0.0590, Val AP: 0.1350, Test AP: 0.1316
Seed: 42, Epoch: 012, Loss: 0.0587, Val AP: 0.1381, Test AP: 0.1366
Seed: 42, Epoch: 013, Loss: 0.0585, Val AP: 0.1388, Test AP: 0.1358
Seed: 42, Epoch: 014, Loss: 0.0583, Val AP: 0.1465, Test AP: 0.1440
Seed: 42, Epoch: 015, Loss: 0.0582, Val AP: 0.15

### COPooling

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.
    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)
    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])
    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)
    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))
    return out
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i
    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]
    if edge_attr is not None:
        edge_attr = edge_attr[mask]
    return torch.stack([row, col], dim=0), edge_attr
def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = (x > scores_min).nonzero().view(-1)
        return perm
    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')
        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)
        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)
        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]
        return x_perm[batch_perm[mask]]
    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")
class GPR_prop(MessagePassing):
    '''
    propagation class for GPR_GNN
    '''
    def __init__(self, K, alpha, Init, Gamma=None, bias=True, **kwargs):
        super(GPR_prop, self).__init__(aggr='add', **kwargs)
        self.K = K
        self.Init = Init
        self.alpha = alpha
        assert Init in ['SGC', 'PPR', 'NPPR', 'Random', 'WS']
        if Init == 'SGC':
            TEMP = 0.0*np.ones(K+1)
            TEMP[alpha] = 1.0
        elif Init == 'PPR':
            TEMP = alpha*(1-alpha)**np.arange(K+1)
            TEMP[-1] = (1-alpha)**K
        elif Init == 'NPPR':
            TEMP = (alpha)**np.arange(K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'Random':
            bound = np.sqrt(3/(K+1))
            TEMP = np.random.uniform(-bound, bound, K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'WS':
            TEMP = Gamma
        self.temp = Parameter(torch.tensor(TEMP))
    def reset_parameters(self):
        torch.nn.init.zeros_(self.temp)
        for k in range(self.K+1):
            self.temp.data[k] = self.alpha*(1-self.alpha)**k
        self.temp.data[-1] = (1-self.alpha)**self.K
    def forward(self, x, edge_index, edge_weight=None):
        edge_index, norm = gcn_norm(
            edge_index, edge_weight, num_nodes=x.size(0), dtype=x.dtype)
        hidden = x*(self.temp[0])
        for k in range(self.K):
            x = self.propagate(edge_index, x=x, norm=norm)
            gamma = self.temp[k+1]
            hidden = hidden + gamma*x
        return hidden
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def __repr__(self):
        return '{}(K={}, temp={})'.format(self.__class__.__name__, self.K,
                                           self.temp)
class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes) 
        edge_index = edge_index.type(torch.long)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)
        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        return aggr_out
class graph_attention(torch.nn.Module):
    src_nodes_dim = 0  
    trg_nodes_dim = 1  
    nodes_dim = 0      
    head_dim = 1       
    def __init__(self, num_in_features, num_out_features, num_of_heads, dropout_prob=0.6, log_attention_weights=False):
        super().__init__()
        self.num_of_heads = num_of_heads
        self.num_out_features = num_out_features
        self.linear_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)
        self.scoring_fn_target = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.scoring_fn_source = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.init_params()
    def init_params(self):
        """
        The reason we're using Glorot (aka Xavier uniform) initialization is because it's a default TF initialization:
            https://stackoverflow.com/questions/37350131/what-is-the-default-variable-initializer-in-tensorflow
        The original repo was developed in TensorFlow (TF) and they used the default initialization.
        Feel free to experiment - there may be better initializations depending on your problem.
        """
        nn.init.xavier_uniform_(self.linear_proj.weight)
        nn.init.xavier_uniform_(self.scoring_fn_target)
        nn.init.xavier_uniform_(self.scoring_fn_source)
    def forward(self, x, edge_index):
        in_nodes_features = x  
        num_of_nodes = in_nodes_features.shape[self.nodes_dim]
        nodes_features_proj = self.linear_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)
        scores_source = (nodes_features_proj * self.scoring_fn_source).sum(dim=-1)
        scores_target = (nodes_features_proj * self.scoring_fn_target).sum(dim=-1)
        scores_source_lifted, scores_target_lifted, nodes_features_proj_lifted = self.lift(scores_source, scores_target, nodes_features_proj, edge_index)
        scores_per_edge = scores_source_lifted + scores_target_lifted
        return torch.sigmoid(scores_per_edge)
    def lift(self, scores_source, scores_target, nodes_features_matrix_proj, edge_index):
        """
        Lifts i.e. duplicates certain vectors depending on the edge index.
        One of the tensor dims goes from N -> E (that's where the "lift" comes from).
        """
        src_nodes_index = edge_index[self.src_nodes_dim]
        trg_nodes_index = edge_index[self.trg_nodes_dim]
        scores_source = scores_source.index_select(self.nodes_dim, src_nodes_index)
        scores_target = scores_target.index_select(self.nodes_dim, trg_nodes_index)
        nodes_features_matrix_proj_lifted = nodes_features_matrix_proj.index_select(self.nodes_dim, src_nodes_index)
        return scores_source, scores_target, nodes_features_matrix_proj_lifted
class CoPooling(torch.nn.Module):
    def __init__(self, ratio=0.5, K=0.05, edge_ratio=0.6, nhid=64, alpha=0.1, Init='Random', Gamma=None):
        super(CoPooling, self).__init__()
        self.ratio = ratio
        self.calc_information_score = NodeInformationScore()
        self.edge_ratio = edge_ratio
        self.prop1 = GPR_prop(K, alpha, Init, Gamma)
        score_dim = 32
        self.G_att = graph_attention(num_in_features=nhid, num_out_features=score_dim, num_of_heads=1)
        self.weight = Parameter(torch.Tensor(2*nhid, nhid))
        nn.init.xavier_uniform_(self.weight.data)
        self.bias = Parameter(torch.Tensor(nhid))
        nn.init.zeros_(self.bias.data)
        self.reset_parameters()
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight.data)
        nn.init.zeros_(self.bias.data)
        self.prop1.reset_parameters()
        self.G_att.init_params()
    def forward(self, x, edge_index, edge_attr, batch=None, nodes_index=None, node_attr=None):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        ori_batch = batch.clone()
        device = x.device
        num_nodes = x.shape[0]
        x_cut = self.prop1(x, edge_index) 
        attention = self.G_att(x_cut, edge_index) 
        attention = attention.sum(dim=1) 
        edge_index, attention = add_self_loops(edge_index, attention, 1.0, num_nodes) 
        edge_index_t, attention_t = transpose(edge_index, attention, num_nodes, num_nodes)
        edge_tmp = torch.cat((edge_index, edge_index_t), 1)
        att_tmp = torch.cat((attention, attention_t),0)
        edge_index, attention = coalesce(edge_tmp, att_tmp, num_nodes, num_nodes, 'mean')
        attention_np = attention.cpu().data.numpy()
        cut_val = np.percentile(attention_np, int(100*(1-self.edge_ratio))) 
        attention = attention * (attention >= cut_val) 
        kep_idx = attention > 0.0
        cut_edge_index, cut_edge_attr = edge_index[:, kep_idx], attention[kep_idx]
        x_information_score = self.calc_information_score(x, cut_edge_index, cut_edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)
        perm = topk(score, self.ratio, batch)
        x_topk = x[perm]
        batch = batch[perm]
        if nodes_index is not None:
            nodes_index = nodes_index[perm]
        if node_attr is not None:
            node_attr = node_attr[perm]
        if cut_edge_index is not None or cut_edge_index.nelement() != 0:
            induced_edge_index, induced_edge_attr = filter_adj(cut_edge_index, cut_edge_attr, perm, num_nodes=num_nodes)
        else:
            print('All edges are cut!')
            induced_edge_index, induced_edge_attr = cut_edge_index, cut_edge_attr
        attention_dense = (to_dense_adj(cut_edge_index, edge_attr=cut_edge_attr, max_num_nodes=num_nodes)).squeeze()
        x = F.relu(torch.matmul(torch.cat((x_topk, torch.matmul(attention_dense[perm],x)), 1), self.weight) + self.bias)
        return x, induced_edge_index, perm, induced_edge_attr, batch, nodes_index, node_attr, attention_dense

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_CO(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CO, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CoPooling(ratio=0.9, K=1, edge_ratio=0.6, nhid=512, alpha=0.1, Init='Random', Gamma=1.0)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CoPooling(ratio=0.9, K=1, edge_ratio=0.6, nhid=512, alpha=0.1, Init='Random', Gamma=1.0)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, perm, _, batch, _, _, _ = self.pool1(x, edge_index, edge_attr=edge_attr, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, perm, _, batch, _, _, _ = self.pool2(x, edge_index, edge_attr=edge_attr, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CO(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = HierarchicalGCN_CO(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0621, Val AP: 0.1176, Test AP: 0.1225
Seed: 42, Epoch: 002, Loss: 0.0603, Val AP: 0.1144, Test AP: 0.1190
Seed: 42, Epoch: 003, Loss: 0.0603, Val AP: 0.1173, Test AP: 0.1225
Seed: 42, Epoch: 004, Loss: 0.0603, Val AP: 0.1171, Test AP: 0.1220
Seed: 42, Epoch: 005, Loss: 0.0603, Val AP: 0.1181, Test AP: 0.1231
Seed: 42, Epoch: 006, Loss: 0.0603, Val AP: 0.1183, Test AP: 0.1233
Seed: 42, Epoch: 007, Loss: 0.0603, Val AP: 0.1180, Test AP: 0.1231
Seed: 42, Epoch: 008, Loss: 0.0603, Val AP: 0.1177, Test AP: 0.1232
Seed: 42, Epoch: 009, Loss: 0.0603, Val AP: 0.1182, Test AP: 0.1233
Seed: 42, Epoch: 010, Loss: 0.0602, Val AP: 0.1207, Test AP: 0.1250
Seed: 42, Epoch: 011, Loss: 0.0602, Val AP: 0.1211, Test AP: 0.1267
Seed: 42, Epoch: 012, Loss: 0.0602, Val AP: 0.1209, Test AP: 0.1264
Seed: 42, Epoch: 013, Loss: 0.0601, Val AP: 0.1213, Test AP: 0.1271
Seed: 42, Epoch: 014, Loss: 0.0601, Val AP: 0.1192, Test AP: 0.1246
Seed: 42, Epoch: 015, Loss: 0.0601, Val AP: 0.12

### CGIPooling

In [2]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
@dataclass(init=False)
class SelectOutput:
    r"""The output of the :class:`Select` method, which holds an assignment
    from selected nodes to their respective cluster(s).
    Args:
        node_index (torch.Tensor): The indices of the selected nodes.
        num_nodes (int): The number of nodes.
        cluster_index (torch.Tensor): The indices of the clusters each node in
            :obj:`node_index` is assigned to.
        num_clusters (int): The number of clusters.
        weight (torch.Tensor, optional): A weight vector, denoting the strength
            of the assignment of a node to its cluster. (default: :obj:`None`)
    """
    node_index: Tensor
    num_nodes: int
    cluster_index: Tensor
    num_clusters: int
    weight: Optional[Tensor] = None
    def __init__(
        self,
        node_index: Tensor,
        num_nodes: int,
        cluster_index: Tensor,
        num_clusters: int,
        weight: Optional[Tensor] = None,
    ):
        if node_index.dim() != 1:
            raise ValueError(f"Expected 'node_index' to be one-dimensional "
                             f"(got {node_index.dim()} dimensions)")
        if cluster_index.dim() != 1:
            raise ValueError(f"Expected 'cluster_index' to be one-dimensional "
                             f"(got {cluster_index.dim()} dimensions)")
        if node_index.numel() != cluster_index.numel():
            raise ValueError(f"Expected 'node_index' and 'cluster_index' to "
                             f"hold the same number of values (got "
                             f"{node_index.numel()} and "
                             f"{cluster_index.numel()} values)")
        if weight is not None and weight.dim() != 1:
            raise ValueError(f"Expected 'weight' vector to be one-dimensional "
                             f"(got {weight.dim()} dimensions)")
        if weight is not None and weight.numel() != node_index.numel():
            raise ValueError(f"Expected 'weight' to hold {node_index.numel()} "
                             f"values (got {weight.numel()} values)")
        self.node_index = node_index
        self.num_nodes = num_nodes
        self.cluster_index = cluster_index
        self.num_clusters = num_clusters
        self.weight = weight
class Select(torch.nn.Module):
    r"""An abstract base class for implementing custom node selections as
    described in the `"Understanding Pooling in Graph Neural Networks"
    <https://arxiv.org/abs/1905.05178>`_ paper, which maps the nodes of an
    input graph to supernodes in the coarsened graph.
    Specifically, :class:`Select` returns a :class:`SelectOutput` output, which
    holds a (sparse) mapping :math:`\mathbf{C} \in {[0, 1]}^{N \times C}` that
    assigns selected nodes to one or more of :math:`C` super nodes.
    """
    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        pass
    def forward(self, *args, **kwargs) -> SelectOutput:
        raise NotImplementedError
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}()'
def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.
    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)
    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])
    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)
    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))
    return out
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i
    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]
    if edge_attr is not None:
        edge_attr = edge_attr[mask]
    return torch.stack([row, col], dim=0), edge_attr
def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = (x > scores_min).nonzero().view(-1)
        return perm
    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')
        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)
        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)
        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]
        return x_perm[batch_perm[mask]]
    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(in_channels * 2, in_channels)
        self.fc2 = nn.Linear(in_channels, 1)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.sigmoid(self.fc2(x))
        return x
class CGIPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, non_lin=torch.tanh):
        super(CGIPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.non_lin = non_lin
        self.hidden_dim = in_channels
        self.transform = GraphConv(in_channels, self.hidden_dim)
        self.pp_conv = GraphConv(self.hidden_dim, self.hidden_dim)
        self.np_conv = GraphConv(self.hidden_dim, self.hidden_dim)
        self.positive_pooling = GraphConv(self.hidden_dim, 1)
        self.negative_pooling = GraphConv(self.hidden_dim, 1)
        self.discriminator = Discriminator(self.hidden_dim)
        self.loss_fn = torch.nn.BCELoss()
    def forward(self, x, edge_index, edge_attr=None, batch=None):
        device = x.device  
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x_transform = F.leaky_relu(self.transform(x, edge_index), 0.2)
        x_tp = F.leaky_relu(self.pp_conv(x, edge_index), 0.2)
        x_tn = F.leaky_relu(self.np_conv(x, edge_index), 0.2)
        s_pp = self.positive_pooling(x_tp, edge_index).squeeze()
        s_np = self.negative_pooling(x_tn, edge_index).squeeze()
        perm_positive = topk(s_pp, 1, batch)
        perm_negative = topk(s_np, 1, batch)
        x_pp = x_transform[perm_positive] * self.non_lin(s_pp[perm_positive]).view(-1, 1)
        x_np = x_transform[perm_negative] * self.non_lin(s_np[perm_negative]).view(-1, 1)
        x_pp_readout = gap(x_pp, batch[perm_positive])
        x_np_readout = gap(x_np, batch[perm_negative])
        x_readout = gap(x_transform, batch)
        positive_pair = torch.cat([x_pp_readout, x_readout], dim=1)
        negative_pair = torch.cat([x_np_readout, x_readout], dim=1)
        real = torch.ones(positive_pair.shape[0], device=device)  
        fake = torch.zeros(negative_pair.shape[0], device=device)  
        score = (s_pp - s_np)
        perm = topk(score, self.ratio, batch)
        x = x_transform[perm] * self.non_lin(score[perm]).view(-1, 1)
        batch = batch[perm]
        filter_edge_index, filter_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        return x, filter_edge_index, filter_edge_attr, batch, perm

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CGIPool(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CGIPool(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, perm = self.pool1(x, edge_index, None, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, perm = self.pool2(x, edge_index, None, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)
    model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0649, Val AP: 0.1149, Test AP: 0.1142
Seed: 42, Epoch: 002, Loss: 0.0608, Val AP: 0.1208, Test AP: 0.1222
Seed: 42, Epoch: 003, Loss: 0.0603, Val AP: 0.1208, Test AP: 0.1218
Seed: 42, Epoch: 004, Loss: 0.0602, Val AP: 0.1204, Test AP: 0.1213
Seed: 42, Epoch: 005, Loss: 0.0602, Val AP: 0.1208, Test AP: 0.1217
Seed: 42, Epoch: 006, Loss: 0.0602, Val AP: 0.1205, Test AP: 0.1213
Seed: 42, Epoch: 007, Loss: 0.0602, Val AP: 0.1207, Test AP: 0.1213
Seed: 42, Epoch: 008, Loss: 0.0602, Val AP: 0.1206, Test AP: 0.1214
Seed: 42, Epoch: 009, Loss: 0.0602, Val AP: 0.1208, Test AP: 0.1215
Seed: 42, Epoch: 010, Loss: 0.0602, Val AP: 0.1208, Test AP: 0.1215
Seed: 42, Epoch: 011, Loss: 0.0602, Val AP: 0.1210, Test AP: 0.1218
Seed: 42, Epoch: 012, Loss: 0.0603, Val AP: 0.1210, Test AP: 0.1217
Seed: 42, Epoch: 013, Loss: 0.0602, Val AP: 0.1206, Test AP: 0.1215
Seed: 42, Epoch: 014, Loss: 0.0602, Val AP: 0.1212, Test AP: 0.1218
Seed: 42, Epoch: 015, Loss: 0.0602, Val AP: 0.12

### KMISPooling

In [6]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from typing import Callable, Optional, Tuple, Union
import torch
from torch.nn import Module
from torch_scatter import scatter, scatter_add, scatter_min
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]
import torch
from torch_scatter import scatter_max, scatter_min
from torch_geometric.typing import Adj, OptTensor, SparseTensor, Tensor
from typing import Callable, Optional, Tuple, Union
import torch
from torch.nn import Module
from torch_scatter import scatter, scatter_add, scatter_min
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]
import torch
from torch_scatter import scatter_max, scatter_min
from torch_geometric.typing import Adj, OptTensor, SparseTensor, Tensor
def maximal_independent_set(edge_index: Adj, k: int = 1,
                            perm: OptTensor = None) -> Tensor:
    r"""Returns a Maximal :math:`k`-Independent Set of a graph, i.e., a set of
    nodes (as a :class:`ByteTensor`) such that none of them are :math:`k`-hop
    neighbors, and any node in the graph has a :math:`k`-hop neighbor in the
    returned set.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method follows `Blelloch's Alogirithm
    <https://arxiv.org/abs/1202.3205>`_ for :math:`k = 1`, and its
    generalization by `Bacciu et al. <https://arxiv.org/abs/2208.03523>`_ for
    higher values of :math:`k`.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: :class:`ByteTensor`
    """
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
        device = edge_index.device()
        n = edge_index.size(0)
    else:
        row, col = edge_index[0], edge_index[1]
        device = row.device
        n = edge_index.max().item() + 1
    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)
    mis = torch.zeros(n, dtype=torch.bool, device=device)
    mask = mis.clone()
    min_rank = rank.clone()
    while not mask.all():
        for _ in range(k):
            min_neigh = torch.full_like(min_rank, fill_value=n)
            scatter_min(min_rank[row], col, out=min_neigh)
            torch.minimum(min_neigh, min_rank, out=min_rank)  
        mis = mis | torch.eq(rank, min_rank)
        mask = mis.clone().byte()
        for _ in range(k):
            max_neigh = torch.full_like(mask, fill_value=0)
            scatter_max(mask[row], col, out=max_neigh)
            torch.maximum(max_neigh, mask, out=mask)  
        mask = mask.to(dtype=torch.bool)
        min_rank = rank.clone()
        min_rank[mask] = n
    return mis
def maximal_independent_set_cluster(edge_index: Adj, k: int = 1,
                                    perm: OptTensor = None) -> PairTensor:
    r"""Computes the Maximal :math:`k`-Independent Set (:math:`k`-MIS)
    clustering of a graph, as defined in `"Generalizing Downsampling from
    Regular Data to Graphs" <https://arxiv.org/abs/2208.03523>`_.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method returns both the :math:`k`-MIS and the clustering, where the
    :math:`c`-th cluster refers to the :math:`c`-th element of the
    :math:`k`-MIS.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: (:class:`ByteTensor`, :class:`LongTensor`)
    """
    mis = maximal_independent_set(edge_index=edge_index, k=k, perm=perm)
    n, device = mis.size(0), mis.device
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
    else:
        row, col = edge_index[0], edge_index[1]
    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)
    min_rank = torch.full((n, ), fill_value=n, dtype=torch.long, device=device)
    rank_mis = rank[mis]
    min_rank[mis] = rank_mis
    for _ in range(k):
        min_neigh = torch.full_like(min_rank, fill_value=n)
        scatter_min(min_rank[row], col, out=min_neigh)
        torch.minimum(min_neigh, min_rank, out=min_rank)
    _, clusters = torch.unique(min_rank, return_inverse=True)
    perm = torch.argsort(rank_mis)
    return mis, perm[clusters]
class KMISPooling(Module):
    _heuristics = {None, 'greedy', 'w-greedy'}
    _passthroughs = {None, 'before', 'after'}
    _scorers = {
        'linear',
        'random',
        'constant',
        'canonical',
        'first',
        'last',
    }
    def __init__(self, in_channels: Optional[int] = None, k: int = 1,
                 scorer: Union[Scorer, str] = 'linear',
                 score_heuristic: Optional[str] = 'greedy',
                 score_passthrough: Optional[str] = 'before',
                 aggr_x: Optional[Union[str, Aggregation]] = None,
                 aggr_edge: str = 'sum',
                 aggr_score: Callable[[Tensor, Tensor], Tensor] = torch.mul,
                 remove_self_loops: bool = True) -> None:
        super(KMISPooling, self).__init__()
        assert score_heuristic in self._heuristics, \
            "Unrecognized `score_heuristic` value."
        assert score_passthrough in self._passthroughs, \
            "Unrecognized `score_passthrough` value."
        if not callable(scorer):
            assert scorer in self._scorers, \
                "Unrecognized `scorer` value."
        self.k = k
        self.scorer = scorer
        self.score_heuristic = score_heuristic
        self.score_passthrough = score_passthrough
        self.aggr_x = aggr_x
        self.aggr_edge = aggr_edge
        self.aggr_score = aggr_score
        self.remove_self_loops = remove_self_loops
        if scorer == 'linear':
            assert self.score_passthrough is not None, \
                "`'score_passthrough'` must not be `None`" \
                " when using `'linear'` scorer"
            self.lin = Linear(in_features=in_channels, out_features=1)
    def _apply_heuristic(self, x: Tensor, adj: SparseTensor) -> Tensor:
        if self.score_heuristic is None:
            return x
        row, col, _ = adj.coo()
        x = x.view(-1)
        if self.score_heuristic == 'greedy':
            k_sums = torch.ones_like(x)
        else:
            k_sums = x.clone()
        for _ in range(self.k):
            scatter_add(k_sums[row], col, out=k_sums)
        return x / k_sums
    def _scorer(self, x: Tensor, edge_index: Adj, edge_attr: OptTensor = None,
                batch: OptTensor = None) -> Tensor:
        if self.scorer == 'linear':
            return self.lin(x).sigmoid()
        if self.scorer == 'random':
            return torch.rand((x.size(0), 1), device=x.device)
        if self.scorer == 'constant':
            return torch.ones((x.size(0), 1), device=x.device)
        if self.scorer == 'canonical':
            return -torch.arange(x.size(0), device=x.device).view(-1, 1)
        if self.scorer == 'first':
            return x[..., [0]]
        if self.scorer == 'last':
            return x[..., [-1]]
        return self.scorer(x, edge_index, edge_attr, batch)
    def forward(self, x: Tensor, edge_index: Adj,
                edge_attr: OptTensor = None,
                batch: OptTensor = None) \
            -> Tuple[Tensor, Adj, OptTensor, OptTensor, Tensor, Tensor]:
        """"""
        edge_index = edge_index.long()
        adj, n = edge_index, x.size(0)
        if not isinstance(edge_index, SparseTensor):
            adj = SparseTensor.from_edge_index(edge_index, edge_attr, (n, n))
        score = self._scorer(x, edge_index, edge_attr, batch)
        updated_score = self._apply_heuristic(score, adj)
        perm = torch.argsort(updated_score.view(-1), 0, descending=True)
        mis, cluster = maximal_independent_set_cluster(adj, self.k, perm)
        row, col, val = adj.coo()
        c = mis.sum()
        if val is None:
            val = torch.ones_like(row, dtype=torch.float)
        adj = SparseTensor(row=cluster[row], col=cluster[col], value=val,
                           is_sorted=False,
                           sparse_sizes=(c, c)).coalesce(self.aggr_edge)
        if self.remove_self_loops:
            adj = remove_diag(adj)
        if self.score_passthrough == 'before':
            x = self.aggr_score(x, score)
        if self.aggr_x is None:
            x = x[mis]
        elif isinstance(self.aggr_x, str):
            x = scatter(x, cluster, dim=0, dim_size=mis.sum(),
                        reduce=self.aggr_x)
        else:
            x = self.aggr_x(x, cluster, dim_size=c)
        if self.score_passthrough == 'after':
            x = self.aggr_score(x, score[mis])
        if isinstance(edge_index, SparseTensor):
            edge_index, edge_attr = adj, None
        else:
            row, col, edge_attr = adj.coo()
            edge_index = torch.stack([row, col])
        if batch is not None:
            batch = batch[mis]
        return x, edge_index, edge_attr, batch, mis, cluster
    def __repr__(self):
        if self.scorer == 'linear':
            channels = f"in_channels={self.lin.in_channels}, "
        else:
            channels = ""
        return f'{self.__class__.__name__}({channels}k={self.k})'

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(512, k=1, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(512, k=1, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)
    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0629, Val AP: 0.1179, Test AP: 0.1196
Seed: 42, Epoch: 002, Loss: 0.0602, Val AP: 0.1155, Test AP: 0.1192
Seed: 42, Epoch: 003, Loss: 0.0601, Val AP: 0.1131, Test AP: 0.1162
Seed: 42, Epoch: 004, Loss: 0.0599, Val AP: 0.1199, Test AP: 0.1217
Seed: 42, Epoch: 005, Loss: 0.0596, Val AP: 0.1223, Test AP: 0.1230
Seed: 42, Epoch: 006, Loss: 0.0593, Val AP: 0.1272, Test AP: 0.1281
Seed: 42, Epoch: 007, Loss: 0.0590, Val AP: 0.1327, Test AP: 0.1351
Seed: 42, Epoch: 008, Loss: 0.0587, Val AP: 0.1438, Test AP: 0.1463
Seed: 42, Epoch: 009, Loss: 0.0583, Val AP: 0.1493, Test AP: 0.1524
Seed: 42, Epoch: 010, Loss: 0.0580, Val AP: 0.1426, Test AP: 0.1448
Seed: 42, Epoch: 011, Loss: 0.0577, Val AP: 0.1562, Test AP: 0.1593
Seed: 42, Epoch: 012, Loss: 0.0574, Val AP: 0.1520, Test AP: 0.1550
Seed: 42, Epoch: 013, Loss: 0.0572, Val AP: 0.1563, Test AP: 0.1595
Seed: 42, Epoch: 014, Loss: 0.0569, Val AP: 0.1635, Test AP: 0.1665
Seed: 42, Epoch: 015, Loss: 0.0567, Val AP: 0.16

### GSAPool

In [9]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
import torch
import torch.nn as nn
import numpy as np
from typing import Union, Optional, Callable
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax
import math
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GCNConv, GATConv, ChebConv, GraphConv
def uniform(size, tensor):
    if tensor is not None:
        bound = 1.0 / math.sqrt(size)
        tensor.data.uniform_(-bound, bound)
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def topk(x, ratio, batch, min_score=None, tol=1e-7):
    if min_score is not None:
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = (x > scores_min).nonzero(as_tuple=False).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
             num_nodes.cumsum(dim=0)[:-1]], dim=0)
        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
        dense_x = x.new_full((batch_size * max_num_nodes, ),
                             torch.finfo(x.dtype).min)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)
        _, perm = dense_x.sort(dim=-1, descending=True)
        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)
        if isinstance(ratio, int):
            k = num_nodes.new_full((num_nodes.size(0), ), ratio)
            k = torch.min(k, num_nodes)
        else:
            k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)
        perm = perm[mask]
    return perm
def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i
    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]
    if edge_attr is not None:
        edge_attr = edge_attr[mask]
    return torch.stack([row, col], dim=0), edge_attr
class GSAPool(torch.nn.Module):
    def __init__(self, in_channels, pooling_ratio=0.5, alpha=0.6,
                        min_score=None, multiplier=1,
                        non_linearity=torch.tanh,
                        cus_drop_ratio =0):
        super(GSAPool,self).__init__()
        self.in_channels = in_channels
        self.ratio = pooling_ratio
        self.alpha = alpha
        self.sbtl_layer = GCNConv(in_channels,1)
        self.fbtl_layer = nn.Linear(in_channels, 1)
        self.fusion = GCNConv(in_channels,in_channels)
        self.min_score = min_score
        self.multiplier = multiplier
        self.fusion_flag = 0
        self.non_linearity = non_linearity
        self.dropout = torch.nn.Dropout(cus_drop_ratio)
    def conv_selection(self, conv, in_channels, conv_type=0):
        if(conv_type == 0):
            out_channels = 1
        elif(conv_type == 1):
            out_channels = in_channels
        if(conv == "GCNConv"):
            return GCNConv(in_channels,out_channels)
        elif(conv == "ChebConv"):
            return ChebConv(in_channels,out_channels,1)
        elif(conv == "GCNConv"):
            return GCNConv(in_channels,out_channels)
        elif(conv == "GATConv"):
            return GATConv(in_channels,out_channels, heads=1, concat=True)
        elif(conv == "GraphConv"):
            return GraphConv(in_channels,out_channels)
        else:
            raise ValueError
    def forward(self, x, edge_index, edge_attr=None, batch=None):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x = x.unsqueeze(-1) if x.dim() == 1 else x
        score_s = self.sbtl_layer(x,edge_index).squeeze()
        score_f = self.fbtl_layer(x).squeeze()
        score = score_s*self.alpha + score_f*(1-self.alpha)
        score = score.unsqueeze(-1) if score.dim()==0 else score
        if self.min_score is None:
            score = self.non_linearity(score)
        else:
            score = softmax(score, batch)
        sc = self.dropout(score)
        perm = topk(sc, self.ratio, batch)
        if(self.fusion_flag == 1):
            x = self.fusion(x, edge_index)
        x_ae = x[perm]
        x = x[perm] * score[perm].view(-1, 1)
        x = self.multiplier * x if self.multiplier != 1 else x
        batch = batch[perm]
        edge_index, edge_attr = filter_adj(
            edge_index, edge_attr, perm, num_nodes=score.size(0))
        return x, edge_index, edge_attr, batch, perm, x_ae

In [11]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
class HierarchicalGCN_GSA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_GSA, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = GSAPool(512, pooling_ratio=0.9, alpha = 0.6, cus_drop_ratio = 0)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = GSAPool(512, pooling_ratio=0.9, alpha = 0.6, cus_drop_ratio = 0)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, perm_1, x_ae1 = self.pool1(x, edge_index, None, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, perm_1, x_ae1 = self.pool2(x, edge_index, None, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_GSA(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)
    model = HierarchicalGCN_GSA(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0628, Val AP: 0.1183, Test AP: 0.1179
Seed: 42, Epoch: 002, Loss: 0.0604, Val AP: 0.1160, Test AP: 0.1169
Seed: 42, Epoch: 003, Loss: 0.0601, Val AP: 0.1245, Test AP: 0.1246
Seed: 42, Epoch: 004, Loss: 0.0596, Val AP: 0.1165, Test AP: 0.1168
Seed: 42, Epoch: 005, Loss: 0.0594, Val AP: 0.1340, Test AP: 0.1329
Seed: 42, Epoch: 006, Loss: 0.0591, Val AP: 0.1343, Test AP: 0.1331
Seed: 42, Epoch: 007, Loss: 0.0588, Val AP: 0.1402, Test AP: 0.1383
Seed: 42, Epoch: 008, Loss: 0.0585, Val AP: 0.1444, Test AP: 0.1424
Seed: 42, Epoch: 009, Loss: 0.0582, Val AP: 0.1556, Test AP: 0.1543
Seed: 42, Epoch: 010, Loss: 0.0578, Val AP: 0.1632, Test AP: 0.1615
Seed: 42, Epoch: 011, Loss: 0.0576, Val AP: 0.1514, Test AP: 0.1513
Seed: 42, Epoch: 012, Loss: 0.0574, Val AP: 0.1614, Test AP: 0.1610
Seed: 42, Epoch: 013, Loss: 0.0571, Val AP: 0.1496, Test AP: 0.1485
Seed: 42, Epoch: 014, Loss: 0.0570, Val AP: 0.1703, Test AP: 0.1726
Seed: 42, Epoch: 015, Loss: 0.0567, Val AP: 0.17

### HGPSLPool

In [73]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
import torch
import torch.nn as nn
import numpy as np
from typing import Union, Optional, Callable
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax
import math
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GCNConv, GATConv, ChebConv, GraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.data import Data
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import softmax, dense_to_sparse, add_remaining_self_loops
from torch_scatter import scatter_add
from torch_sparse import spspmm, coalesce
import torch
import torch.nn as nn
from torch.autograd import Function
from torch_scatter import scatter_add, scatter_max
def topk(x, ratio, batch, min_score=None, tol=1e-7):
    if min_score is not None:
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = torch.nonzero(x > scores_min).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
            num_nodes.cumsum(dim=0)[:-1]], dim=0)
        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
        dense_x = x.new_full((batch_size * max_num_nodes, ), -2)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)
        _, perm = dense_x.sort(dim=-1, descending=True)
        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)
        k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)
        perm = perm[mask]
    return perm
def filter_adj(edge_index, edge_weight, perm, num_nodes=None):
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        mask = perm.new_full((num_nodes, ), -1)
        i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
        mask[perm] = i
        row, col = edge_index
        row, col = mask[row], mask[col]
        mask = (row >= 0) & (col >= 0)
        row, col = row[mask], col[mask]
        if edge_weight is not None:
            edge_weight = edge_weight[mask]
        return torch.stack([row, col], dim=0), edge_weight
def scatter_sort(x, batch, fill_value=-1e16):
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
    index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
    index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
    dense_x = x.new_full((batch_size * max_num_nodes,), fill_value)
    dense_x[index] = x
    dense_x = dense_x.view(batch_size, max_num_nodes)
    sorted_x, _ = dense_x.sort(dim=-1, descending=True)
    cumsum_sorted_x = sorted_x.cumsum(dim=-1)
    cumsum_sorted_x = cumsum_sorted_x.view(-1)
    sorted_x = sorted_x.view(-1)
    filled_index = sorted_x != fill_value
    sorted_x = sorted_x[filled_index]
    cumsum_sorted_x = cumsum_sorted_x[filled_index]
    return sorted_x, cumsum_sorted_x
def _make_ix_like(batch):
    num_nodes = scatter_add(batch.new_ones(batch.size(0)), batch, dim=0)
    idx = [torch.arange(1, i + 1, dtype=torch.long, device=batch.device) for i in num_nodes]
    idx = torch.cat(idx, dim=0)
    return idx
def _threshold_and_support(x, batch):
    """Sparsemax building block: compute the threshold
    Args:
        x: input tensor to apply the sparsemax
        batch: group indicators
    Returns:
        the threshold value
    """
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
    sorted_input, input_cumsum = scatter_sort(x, batch)
    input_cumsum = input_cumsum - 1.0
    rhos = _make_ix_like(batch).to(x.dtype)
    support = rhos * sorted_input > input_cumsum
    support_size = scatter_add(support.to(batch.dtype), batch)
    idx = support_size + cum_num_nodes - 1
    mask = idx < 0
    idx[mask] = 0
    tau = input_cumsum.gather(0, idx)
    tau /= support_size.to(x.dtype)
    return tau, support_size
class SparsemaxFunction(Function):
    @staticmethod
    def forward(ctx, x, batch):
        """sparsemax: normalizing sparse transform
        Parameters:
            ctx: context object
            x (Tensor): shape (N, )
            batch: group indicator
        Returns:
            output (Tensor): same shape as input
        """
        max_val, _ = scatter_max(x, batch)
        x -= max_val[batch]
        tau, supp_size = _threshold_and_support(x, batch)
        output = torch.clamp(x - tau[batch], min=0)
        ctx.save_for_backward(supp_size, output, batch)
        return output
    @staticmethod
    def backward(ctx, grad_output):
        supp_size, output, batch = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[output == 0] = 0
        v_hat = scatter_add(grad_input, batch) / supp_size.to(output.dtype)
        grad_input = torch.where(output != 0, grad_input - v_hat[batch], grad_input)
        return grad_input, None
sparsemax = SparsemaxFunction.apply
class Sparsemax(nn.Module):
    def __init__(self):
        super(Sparsemax, self).__init__()
    def forward(self, x, batch):
        return sparsemax(x, batch)
if __name__ == '__main__':
    sparse_attention = Sparsemax()
    input_x = torch.tensor([1.7301, 0.6792, -1.0565, 1.6614, -0.3196, -0.7790, -0.3877, -0.4943, 0.1831, -0.0061])
    input_batch = torch.cat([torch.zeros(4, dtype=torch.long), torch.ones(6, dtype=torch.long)], dim=0)
    res = sparse_attention(input_x, input_batch)
    print(res)
class TwoHopNeighborhood(object):
    def __call__(self, data):
        edge_index, edge_attr = data.edge_index, data.edge_attr
        n = data.num_nodes
        fill = 1e16
        value = edge_index.new_full((edge_index.size(1),), fill, dtype=torch.float)
        index, value = spspmm(edge_index, value, edge_index, value, n, n, n, True)
        edge_index = torch.cat([edge_index, index], dim=1)
        if edge_attr is None:
            data.edge_index, _ = coalesce(edge_index, None, n, n)
        else:
            value = value.view(-1, *[1 for _ in range(edge_attr.dim() - 1)])
            value = value.expand(-1, *list(edge_attr.size())[1:])
            edge_attr = torch.cat([edge_attr, value], dim=0)
            data.edge_index, edge_attr = coalesce(edge_index, edge_attr, n, n, op='min', fill_value=fill)
            edge_attr[edge_attr >= fill] = 0
            data.edge_attr = edge_attr
        return data
    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)
class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels, cached=False, bias=True, **kwargs):
        super(GCN, self).__init__(aggr='add', **kwargs)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        nn.init.xavier_uniform_(self.weight.data)
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
            nn.init.zeros_(self.bias.data)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    def reset_parameters(self):
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight=None):
        x = torch.matmul(x, self.weight)
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out
    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels, self.out_channels)
class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes) 
        edge_index = edge_index.type(torch.long)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)
        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        return aggr_out
class HGPSLPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2):
        super(HGPSLPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.sample = sample
        self.sparse = sparse
        self.sl = sl
        self.negative_slop = negative_slop
        self.lamb = lamb
        self.att = Parameter(torch.Tensor(1, self.in_channels * 2))
        nn.init.xavier_uniform_(self.att.data)
        self.sparse_attention = Sparsemax()
        self.neighbor_augment = TwoHopNeighborhood()
        self.calc_information_score = NodeInformationScore()
    def forward(self, x, edge_index, edge_attr, batch):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x_information_score = self.calc_information_score(x, edge_index, edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)
        original_x = x
        perm = topk(score, self.ratio, batch)
        x = x[perm]
        batch = batch[perm]
        induced_edge_index, induced_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        if self.sl is False:
            return x, induced_edge_index, induced_edge_attr, batch
        if self.sample:
            k_hop = 3
            if edge_attr is None:
                edge_attr = torch.ones((edge_index.size(1),), dtype=torch.float, device=edge_index.device)
            hop_data = Data(x=original_x, edge_index=edge_index, edge_attr=edge_attr)
            for _ in range(k_hop - 1):
                hop_data = self.neighbor_augment(hop_data)
            hop_edge_index = hop_data.edge_index
            hop_edge_attr = hop_data.edge_attr
            new_edge_index, new_edge_attr = filter_adj(hop_edge_index, hop_edge_attr, perm, num_nodes=score.size(0))
            new_edge_index, new_edge_attr = add_remaining_self_loops(new_edge_index, new_edge_attr, 0, x.size(0))
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop) + new_edge_attr * self.lamb
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            adj[row, col] = weights
            new_edge_index, weights = dense_to_sparse(adj)
            row, col = new_edge_index
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, x.size(0))
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            del adj
            torch.cuda.empty_cache()
        else:
            if edge_attr is None:
                induced_edge_attr = torch.ones((induced_edge_index.size(1),), dtype=x.dtype,
                                               device=induced_edge_index.device)
            num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
            shift_cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
            cum_num_nodes = num_nodes.cumsum(dim=0)
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            for idx_i, idx_j in zip(shift_cum_num_nodes, cum_num_nodes):
                adj[idx_i:idx_j, idx_i:idx_j] = 1.0
            new_edge_index, _ = dense_to_sparse(adj)
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop)
            adj[row, col] = weights
            induced_row, induced_col = induced_edge_index
            adj[induced_row, induced_col] += induced_edge_attr * self.lamb
            weights = adj[row, col]
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, num_nodes=x.size(0))
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            del adj
            torch.cuda.empty_cache()
        return x, new_edge_index, new_edge_attr, batch

tensor([0.5344, 0.0000, 0.0000, 0.4656, 0.0613, 0.0000, 0.0000, 0.0000, 0.5640,
        0.3748])


In [76]:
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_HGPSL, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, improved = True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, improved = True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv3 = GCNConv(hidden_channels, out_channels, improved = True)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        edge_attr=None
        init_x = data.x
        x = x.float()
        init_x = init_x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool1(x, edge_index, edge_attr=edge_attr, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool2(x, edge_index, edge_attr=edge_attr, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        out = model(data)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=512,out_channels=256, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0612, Val AP: 0.1202, Test AP: 0.1205
Seed: 42, Epoch: 002, Loss: 0.0599, Val AP: 0.1202, Test AP: 0.1199
Seed: 42, Epoch: 003, Loss: 0.0594, Val AP: 0.1389, Test AP: 0.1392
Seed: 42, Epoch: 004, Loss: 0.0590, Val AP: 0.1405, Test AP: 0.1412
Seed: 42, Epoch: 005, Loss: 0.0587, Val AP: 0.1402, Test AP: 0.1404
Seed: 42, Epoch: 006, Loss: 0.0584, Val AP: 0.1404, Test AP: 0.1393
Seed: 42, Epoch: 007, Loss: 0.0581, Val AP: 0.1557, Test AP: 0.1555
Seed: 42, Epoch: 008, Loss: 0.0578, Val AP: 0.1526, Test AP: 0.1530
Seed: 42, Epoch: 009, Loss: 0.0577, Val AP: 0.1616, Test AP: 0.1607
Seed: 42, Epoch: 010, Loss: 0.0576, Val AP: 0.1534, Test AP: 0.1524
Seed: 42, Epoch: 011, Loss: 0.0573, Val AP: 0.1612, Test AP: 0.1602
Seed: 42, Epoch: 012, Loss: 0.0571, Val AP: 0.1707, Test AP: 0.1697
Seed: 42, Epoch: 013, Loss: 0.0569, Val AP: 0.1798, Test AP: 0.1793
Seed: 42, Epoch: 014, Loss: 0.0568, Val AP: 0.1805, Test AP: 0.1781
Seed: 42, Epoch: 015, Loss: 0.0566, Val AP: 0.17

### AsymCheegerCutPooling

In [2]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 100
dataset_dense = PygGraphPropPredDataset(root='/data/XXX/Pooling', name="ogbg-molpcba", transform=T.ToSparseTensor())
evaluator = Evaluator("ogbg-molpcba")
eval_metric = dataset_dense.eval_metric
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
dataset_dense = dataset_dense.shuffle()
num_total = len(dataset_dense)
num_train = int(num_total * train_ratio)
num_val = int(num_total * val_ratio)
num_test = num_total - num_train - num_val
train_dataset = dataset_dense[:num_train]
val_dataset = dataset_dense[num_train:num_train + num_val]
test_dataset = dataset_dense[num_train + num_val:]
train_loader = DenseDataLoader(train_dataset, batch_size=2048, shuffle=True)
valid_loader = DenseDataLoader(val_dataset, batch_size=2048, shuffle=False)
test_loader = DenseDataLoader(test_dataset, batch_size=2048, shuffle=False)

In [3]:
from typing import List, Optional, Tuple, Union
import math
import torch
from torch import Tensor
from torch_geometric.nn.models.mlp import Linear
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import BatchNorm
class AsymCheegerCutPool(torch.nn.Module):
    r"""
    The asymmetric cheeger cut pooling layer from the `"Total Variation Graph Neural Networks"
    <https://arxiv.org/abs/2211.06218>`_ paper.
    Args:
        k (int):
            Number of clusters or output nodes
        mlp_channels (int, list of int):
            Number of hidden units for each hidden layer in the MLP used to
            compute cluster assignments. First integer must match the number
            of input channels.
        mlp_activation (any):
            Activation function between hidden layers of the MLP.
            Must be compatible with `torch_geometric.nn.resolver`.
        return_selection (bool):
            Whether to return selection matrix. Cannot not  be False
            if `return_pooled_graph` is False. (default: :obj:`False`)
        return_pooled_graph (bool):
            Whether to return pooled node features and adjacency.
            Cannot be False if `return_selection` is False. (default: :obj:`True`)
        bias (bool):
            whether to add a bias term to the MLP layers. (default: :obj:`True`)
        totvar_coeff (float):
            Coefficient for graph total variation loss component. (default: :obj:`1.0`)
        balance_coeff (float):
            Coefficient for asymmetric norm loss component. (default: :obj:`1.0`)
    """
    def __init__(self,
                 k: int,
                 mlp_channels: Union[int, List[int]],
                 mlp_activation="relu",
                 return_selection: bool = False,
                 return_pooled_graph: bool = True,
                 bias: bool = True,
                 totvar_coeff: float = 1.0,
                 balance_coeff: float = 1.0,
                 ):
        super().__init__()
        if not return_selection and not return_pooled_graph:
            raise ValueError("return_selection and return_pooled_graph can not both be False")
        if isinstance(mlp_channels, int):
            mlp_channels = [mlp_channels]
        act = activation_resolver(mlp_activation)
        in_channels = mlp_channels[0]
        self.mlp = torch.nn.Sequential()
        for channels in mlp_channels[1:]:
            self.mlp.append(Linear(in_channels, channels, bias=bias))
            in_channels = channels
            self.mlp.append(act)
        self.mlp.append(Linear(in_channels, k))
        self.k = k
        self.return_selection = return_selection
        self.return_pooled_graph = return_pooled_graph
        self.totvar_coeff = totvar_coeff
        self.balance_coeff = balance_coeff
        self.reset_parameters()
    def reset_parameters(self):
        for layer in self.mlp:
            if isinstance(layer, Linear):
                torch.nn.init.xavier_uniform(layer.weight)
                torch.nn.init.zeros_(layer.bias)
    def forward(
        self,
        x: Tensor,
        adj: Tensor,
        mask: Optional[Tensor] = None,
    ) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
        r"""
        Args:
            x (Tensor):
                Node feature tensor :math:`\mathbf{X} \in \mathbb{R}^{B \times N \times F}`
                with batch-size :math:`B`, (maximum) number of nodes :math:`N` for each graph,
                and feature dimension :math:`F`. Note that the cluster assignment matrix
                :math:`\mathbf{S} \in \mathbb{R}^{B \times N \times C}` is
                being created within this method.
            adj (Tensor):
                Adjacency tensor :math:`\mathbf{A} \in \mathbb{R}^{B \times N \times N}`.
            mask (BoolTensor, optional):
                Mask matrix :math:`\mathbf{M} \in {\{ 0, 1 \}}^{B \times N}`
                indicating the valid nodes for each graph. (default: :obj:`None`)
        :rtype: (:class:`Tensor`, :class:`Tensor`, :class:`Tensor`,
            :class:`Tensor`, :class:`Tensor`, :class:`Tensor`)
        """
        x = x.unsqueeze(0) if x.dim() == 2 else x
        adj = adj.unsqueeze(0) if adj.dim() == 2 else adj
        s = self.mlp(x)
        s = torch.softmax(s, dim=-1)
        batch_size, n_nodes, _ = x.size()
        if mask is not None:
            mask = mask.view(batch_size, n_nodes, 1).to(x.dtype)
            x, s = x * mask, s * mask
        if self.return_pooled_graph:
            x_pool = torch.matmul(s.transpose(1, 2), x)
            adj_pool = torch.matmul(torch.matmul(s.transpose(1, 2), adj), s)
        tv_loss = self.totvar_coeff*torch.mean(self.totvar_loss(adj, s))
        bal_loss = self.balance_coeff*torch.mean(self.balance_loss(s))
        if self.return_selection and self.return_pooled_graph:
            return s, x_pool, adj_pool, tv_loss, bal_loss
        elif self.return_selection and not self.return_pooled_graph:
            return s, tv_loss, bal_loss
        else:
            return x_pool, adj_pool, tv_loss, bal_loss
    def totvar_loss(self, adj, s):
        l1_norm = torch.sum(torch.abs(s[..., None, :] - s[:, None, ...]), dim=-1)
        loss = torch.sum(adj * l1_norm, dim=(-1, -2))
        n_edges = torch.count_nonzero(adj, dim=(-1, -2))
        loss *= 1 / (2 * n_edges)
        return loss
    def balance_loss(self, s):
        n_nodes = s.size()[-2]
        idx = int(math.floor(n_nodes / self.k))
        quant = torch.sort(s, dim=-2, descending=True)[0][:, idx, :] 
        loss = s - torch.unsqueeze(quant, dim=1)
        loss = (loss >= 0) * (self.k - 1) * loss + (loss < 0) * loss * -1
        loss = torch.sum(loss, dim=(-1, -2)) 
        loss = 1 / (n_nodes * (self.k - 1)) * (n_nodes * (self.k - 1) - loss)
        return loss

In [4]:
from torch_geometric.nn import SAGEConv
import os.path as osp
import time
from math import ceil
from torch_geometric.data import Data, Batch
from torch_geometric.utils import to_dense_batch
from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
N = 150
mp_layers = 1
mp_channels = 64
mp_activation = "relu"
delta_coeff = 2.0
mlp_hidden_layers = 1
mlp_hidden_channels = 128
mlp_activation = "relu"
totvar_coeff = 0.5
balance_coeff = 0.5
epochs = 100
batch_size = 16
learning_rate = 5e-4
l2_reg_val = 0
patience = 10
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, normalize)
        self.conv3 = GCNConv(hidden_channels, out_channels, normalize)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def forward(self, x, adj_t, mask=None):
        adj_t = adj_t.to(torch.float32)
        x = self.conv1(x, adj_t, mask).relu()
        x = self.conv2(x, adj_t, mask).relu()
        x = self.conv3(x, adj_t, mask).relu()
        if self.lin is not None:
            x = self.lin(x).relu()
        return x
class Net_AsymCheegerCut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = GCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = GCNConv(64, 64)
        self.gnn3_embed = GCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 128)
        self.lin2 = torch.nn.Linear(128, 128)
        self.pool1 = AsymCheegerCutPool(int(N//2),
                           mlp_channels=[mp_channels] +
                                [mlp_hidden_channels for _ in range(mlp_hidden_layers)],
                           mlp_activation=mlp_activation,
                           totvar_coeff=totvar_coeff,
                           balance_coeff=balance_coeff,
                           return_selection=False,
                           return_pooled_graph=True)
        self.pool2 = AsymCheegerCutPool(int(N//2),
                           mlp_channels=[mp_channels] +
                                [mlp_hidden_channels for _ in range(mlp_hidden_layers)],
                           mlp_activation=mlp_activation,
                           totvar_coeff=totvar_coeff,
                           balance_coeff=balance_coeff,
                           return_selection=False,
                           return_pooled_graph=True)
    def forward(self, x, adj_t, batch):
        adj_t = adj_t.to(torch.float32)
        s = self.gnn1_pool(x, adj_t)
        x = self.gnn1_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj_t, tv1, bal1 = self.pool1(x, adj_t_dense)
        s = self.gnn2_pool(x, adj_t)
        x = self.gnn2_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj_t, tv1, bal1 = self.pool2(x, adj_t_dense, mask=None)
        x = self.gnn3_embed(x, adj_t)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net_AsymCheegerCut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y 
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = Net_AsymCheegerCut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

KeyboardInterrupt: 

### DifferencePool

In [5]:
from torch_geometric.nn import SAGEConv
import os.path as osp
import time
from math import ceil
from torch_geometric.data import Data, Batch
from torch_geometric.utils import to_dense_batch
from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, normalize)
        self.conv3 = GCNConv(hidden_channels, out_channels, normalize)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def forward(self, x, adj_t, mask=None):
        adj_t = adj_t.to(torch.float32)
        x = self.conv1(x, adj_t, mask).relu()
        x = self.conv2(x, adj_t, mask).relu()
        x = self.conv3(x, adj_t, mask).relu()
        if self.lin is not None:
            x = self.lin(x).relu()
        return x
class Net_Diff(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = GCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = GCNConv(64, 64)
        self.gnn3_embed = GCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, x, adj_t, batch):
        adj_t = adj_t.to(torch.float32)
        s = self.gnn1_pool(x, adj_t)
        x = self.gnn1_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj_t, tv1, bal1 = dense_diff_pool(x, adj_t_dense, s)
        s = self.gnn2_pool(x, adj_t)
        x = self.gnn2_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj_t, tv1, bal1 = dense_diff_pool(x, adj_t_dense)
        x = self.gnn3_embed(x, adj_t)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net_Diff().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y 
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = Net_Diff().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0608, Val AP: 0.1209, Test AP: 0.1243
Seed: 42, Epoch: 002, Loss: 0.0598, Val AP: 0.1313, Test AP: 0.1376
Seed: 42, Epoch: 003, Loss: 0.0593, Val AP: 0.1331, Test AP: 0.1386
Seed: 42, Epoch: 004, Loss: 0.0590, Val AP: 0.1361, Test AP: 0.1400
Seed: 42, Epoch: 005, Loss: 0.0586, Val AP: 0.1411, Test AP: 0.1452
Seed: 42, Epoch: 006, Loss: 0.0583, Val AP: 0.1555, Test AP: 0.1612
Seed: 42, Epoch: 007, Loss: 0.0580, Val AP: 0.1575, Test AP: 0.1617
Seed: 42, Epoch: 008, Loss: 0.0577, Val AP: 0.1665, Test AP: 0.1697
Seed: 42, Epoch: 009, Loss: 0.0574, Val AP: 0.1764, Test AP: 0.1792
Seed: 42, Epoch: 010, Loss: 0.0572, Val AP: 0.1623, Test AP: 0.1645
Seed: 42, Epoch: 011, Loss: 0.0569, Val AP: 0.1792, Test AP: 0.1819
Seed: 42, Epoch: 012, Loss: 0.0567, Val AP: 0.1696, Test AP: 0.1729
Seed: 42, Epoch: 013, Loss: 0.0565, Val AP: 0.1593, Test AP: 0.1621
Seed: 42, Epoch: 014, Loss: 0.0562, Val AP: 0.1826, Test AP: 0.1846
Seed: 42, Epoch: 015, Loss: 0.0561, Val AP: 0.18

### MincutPool

In [8]:
from torch_geometric.nn import SAGEConv
import os.path as osp
import time
from math import ceil
from torch_geometric.data import Data, Batch
from torch_geometric.utils import to_dense_batch
from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_mincut_pool
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, normalize)
        self.conv3 = GCNConv(hidden_channels, out_channels, normalize)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def forward(self, x, adj_t, mask=None):
        adj_t = adj_t.to(torch.float32)
        x = self.conv1(x, adj_t, mask).relu()
        x = self.conv2(x, adj_t, mask).relu()
        x = self.conv3(x, adj_t, mask).relu()
        if self.lin is not None:
            x = self.lin(x).relu()
        return x
class Net_Mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = GCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = GCNConv(64, 64)
        self.gnn3_embed = GCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, x, adj_t, batch):
        adj_t = adj_t.to(torch.float32)
        s = self.gnn1_pool(x, adj_t)
        x = self.gnn1_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj, l1, e1 = dense_mincut_pool(x, adj, s, temp=1.8)
        s = self.gnn2_pool(x, adj_t)
        x = self.gnn2_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=1.8)
        x = self.gnn3_embed(x, adj_t)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net_Mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y 
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = Net_Mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0607, Val AP: 0.1209, Test AP: 0.1236
Seed: 42, Epoch: 002, Loss: 0.0597, Val AP: 0.1313, Test AP: 0.1354
Seed: 42, Epoch: 003, Loss: 0.0591, Val AP: 0.1362, Test AP: 0.1396
Seed: 42, Epoch: 004, Loss: 0.0586, Val AP: 0.1409, Test AP: 0.1449
Seed: 42, Epoch: 005, Loss: 0.0582, Val AP: 0.1476, Test AP: 0.1531
Seed: 42, Epoch: 006, Loss: 0.0579, Val AP: 0.1485, Test AP: 0.1526
Seed: 42, Epoch: 007, Loss: 0.0575, Val AP: 0.1663, Test AP: 0.1708
Seed: 42, Epoch: 008, Loss: 0.0572, Val AP: 0.1612, Test AP: 0.1642
Seed: 42, Epoch: 009, Loss: 0.0569, Val AP: 0.1615, Test AP: 0.1645
Seed: 42, Epoch: 010, Loss: 0.0567, Val AP: 0.1700, Test AP: 0.1732
Seed: 42, Epoch: 011, Loss: 0.0564, Val AP: 0.1797, Test AP: 0.1819
Seed: 42, Epoch: 012, Loss: 0.0563, Val AP: 0.1785, Test AP: 0.1805
Seed: 42, Epoch: 013, Loss: 0.0561, Val AP: 0.1845, Test AP: 0.1876
Seed: 42, Epoch: 014, Loss: 0.0559, Val AP: 0.1881, Test AP: 0.1886
Seed: 42, Epoch: 015, Loss: 0.0557, Val AP: 0.19

### DMoNPool

In [5]:
from torch_geometric.nn import SAGEConv
import os.path as osp
import time
from math import ceil
from torch_geometric.data import Data, Batch
from torch_geometric.utils import to_dense_batch
from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
from torch_geometric.nn import DenseGCNConv, DMoNPooling
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_mincut_pool
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, normalize)
        self.conv3 = GCNConv(hidden_channels, out_channels, normalize)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def forward(self, x, adj_t, mask=None):
        adj_t = adj_t.to(torch.float32)
        x = self.conv1(x, adj_t, mask).relu()
        x = self.conv2(x, adj_t, mask).relu()
        x = self.conv3(x, adj_t, mask).relu()
        if self.lin is not None:
            x = self.lin(x).relu()
        return x
class Net_DMoN(torch.nn.Module):
    def __init__(self, in_channels=dataset_dense.num_features, out_channels=64, hidden_channels=64):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = GCNConv(dataset_dense.num_features, 64)
        self.pool1 = DMoNPooling([hidden_channels, hidden_channels], 8)
        self.pool2 = DMoNPooling([hidden_channels, hidden_channels], 8)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = GCNConv(64, 64)
        self.gnn3_embed = GCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, x, adj_t, batch):
        adj_t = adj_t.to(torch.float32)
        s = self.gnn1_pool(x, adj_t)
        x = self.gnn1_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            _, x, adj, sp1, o1, c1 = self.pool1(x, adj)
        s = self.gnn2_pool(x, adj_t)
        x = self.gnn2_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            _, x, adj, sp2, o2, c2 = self.pool2(x, adj)
        x = self.gnn3_embed(x, adj_t)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net_DMoN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y 
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = Net_DMoN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0609, Val AP: 0.1182, Test AP: 0.1182
Seed: 42, Epoch: 002, Loss: 0.0599, Val AP: 0.1189, Test AP: 0.1200
Seed: 42, Epoch: 003, Loss: 0.0594, Val AP: 0.1316, Test AP: 0.1318
Seed: 42, Epoch: 004, Loss: 0.0590, Val AP: 0.1323, Test AP: 0.1325
Seed: 42, Epoch: 005, Loss: 0.0587, Val AP: 0.1345, Test AP: 0.1350
Seed: 42, Epoch: 006, Loss: 0.0584, Val AP: 0.1472, Test AP: 0.1472
Seed: 42, Epoch: 007, Loss: 0.0581, Val AP: 0.1544, Test AP: 0.1543
Seed: 42, Epoch: 008, Loss: 0.0578, Val AP: 0.1602, Test AP: 0.1610
Seed: 42, Epoch: 009, Loss: 0.0575, Val AP: 0.1625, Test AP: 0.1619
Seed: 42, Epoch: 010, Loss: 0.0573, Val AP: 0.1672, Test AP: 0.1662
Seed: 42, Epoch: 011, Loss: 0.0571, Val AP: 0.1719, Test AP: 0.1720
Seed: 42, Epoch: 012, Loss: 0.0568, Val AP: 0.1757, Test AP: 0.1756
Seed: 42, Epoch: 013, Loss: 0.0566, Val AP: 0.1708, Test AP: 0.1718
Seed: 42, Epoch: 014, Loss: 0.0564, Val AP: 0.1683, Test AP: 0.1695
Seed: 42, Epoch: 015, Loss: 0.0563, Val AP: 0.18

### JustbalancePool

In [8]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 100
dataset_dense = PygGraphPropPredDataset(root='/data/XXX/Pooling', name="ogbg-molpcba", transform=T.ToSparseTensor())
evaluator = Evaluator("ogbg-molpcba")
eval_metric = dataset_dense.eval_metric
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
dataset_dense = dataset_dense.shuffle()
num_total = len(dataset_dense)
num_train = int(num_total * train_ratio)
num_val = int(num_total * val_ratio)
num_test = num_total - num_train - num_val
train_dataset = dataset_dense[:num_train]
val_dataset = dataset_dense[num_train:num_train + num_val]
test_dataset = dataset_dense[num_train + num_val:]
train_loader = DenseDataLoader(train_dataset, batch_size=2048, shuffle=True)
valid_loader = DenseDataLoader(val_dataset, batch_size=2048, shuffle=False)
test_loader = DenseDataLoader(test_dataset, batch_size=2048, shuffle=False)

In [7]:
import torch
from torch_geometric.nn import SAGEConv
import os.path as osp
import time
from math import ceil
from torch_geometric.data import Data, Batch
from torch_geometric.utils import to_dense_batch
from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch.nn import Linear, ReLU, Dropout, LayerNorm, Module, ModuleList
from sklearn.metrics import average_precision_score
from torch_geometric.nn import DenseGCNConv, DMoNPooling
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_mincut_pool
EPS = 1e-15
def just_balance_pool(x, adj, s, mask=None, normalize=True):
    r"""The Just Balance pooling operator from the `"Simplifying Clustering with
    Graph Neural Networks" <https://arxiv.org/abs/2207.08779>`_ paper
    .. math::
        \mathbf{X}^{\prime} &= {\mathrm{softmax}(\mathbf{S})}^{\top} \cdot
        \mathbf{X}
        \mathbf{A}^{\prime} &= {\mathrm{softmax}(\mathbf{S})}^{\top} \cdot
        \mathbf{A} \cdot \mathrm{softmax}(\mathbf{S})
    based on dense learned assignments :math:`\mathbf{S} \in \mathbb{R}^{B
    \times N \times C}`.
    Returns the pooled node feature matrix, the coarsened and symmetrically
    normalized adjacency matrix and the following auxiliary objective:
    .. math::
        \mathcal{L} = - {\mathrm{Tr}(\sqrt{\mathbf{S}^{\top} \mathbf{S}})}
    Args:
        x (Tensor): Node feature tensor :math:`\mathbf{X} \in \mathbb{R}^{B \times N \times F}`
            with batch-size :math:`B`, (maximum) number of nodes :math:`N`
            for each graph, and feature dimension :math:`F`.
        adj (Tensor): Symmetrically normalized adjacency tensor
            :math:`\mathbf{A} \in \mathbb{R}^{B \times N \times N}`.
        s (Tensor): Assignment tensor :math:`\mathbf{S} \in \mathbb{R}^{B \times N \times C}`
            with number of clusters :math:`C`. The softmax does not have to be
            applied beforehand, since it is executed within this method.
        mask (BoolTensor, optional): Mask matrix
            :math:`\mathbf{M} \in {\{ 0, 1 \}}^{B \times N}` indicating
            the valid nodes for each graph. (default: :obj:`None`)
    :rtype: (:class:`Tensor`, :class:`Tensor`, :class:`Tensor`,
        :class:`Tensor`)
    """
    x = x.unsqueeze(0) if x.dim() == 2 else x
    adj = adj.unsqueeze(0) if adj.dim() == 2 else adj
    s = s.unsqueeze(0) if s.dim() == 2 else s
    (batch_size, num_nodes, _), k = x.size(), s.size(-1)
    s = torch.softmax(s, dim=-1)
    if mask is not None:
        mask = mask.view(batch_size, num_nodes, 1).to(x.dtype)
        x, s = x * mask, s * mask
    out = torch.matmul(s.transpose(1, 2), x)
    out_adj = torch.matmul(torch.matmul(s.transpose(1, 2), adj), s)
    ss = torch.matmul(s.transpose(1, 2), s)
    ss_sqrt = torch.sqrt(ss + EPS)
    loss = torch.mean(-_rank3_trace(ss_sqrt))
    if normalize:
        loss = loss / torch.sqrt(torch.tensor(num_nodes * k))
    ind = torch.arange(k, device=out_adj.device)
    out_adj[:, ind, ind] = 0
    d = torch.einsum('ijk->ij', out_adj)
    d = torch.sqrt(d)[:, None] + EPS
    out_adj = (out_adj / d) / d.transpose(1, 2)
    return out, out_adj, loss
def _rank3_trace(x):
    return torch.einsum('ijj->i', x)

In [18]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, normalize)
        self.conv3 = GCNConv(hidden_channels, out_channels, normalize)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def forward(self, x, adj_t, mask=None):
        adj_t = adj_t.to(torch.float32)
        x = self.conv1(x, adj_t, mask).relu()
        x = self.conv2(x, adj_t, mask).relu()
        x = self.conv3(x, adj_t, mask).relu()
        if self.lin is not None:
            x = self.lin(x).relu()
        return x
class Net_just_balance_pool(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = GCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = GCNConv(64, 64)
        self.gnn3_embed = GCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 128)
        self.lin2 = torch.nn.Linear(128, 128)
    def forward(self, x, adj_t, batch):
        adj_t = adj_t.to(torch.float32)
        s = self.gnn1_pool(x, adj_t)
        x = self.gnn1_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj_t, tv1, bal1 = just_balance_pool(x, adj_t_dense, s)
        s = self.gnn2_pool(x, adj_t)
        x = self.gnn2_embed(x, adj_t)
        x = F.relu(x)
        if isinstance(adj_t, SparseTensor):
            adj_t_dense = adj_t.to_dense()
            adj_t_dense = adj_t_dense.unsqueeze(0) if adj_t_dense.dim() == 2 else adj_t_dense
        else:
            adj_t_dense = adj_t.unsqueeze(0) if adj_t.dim() == 2 else adj_t
            x, adj_t, tv1, bal1 = just_balance_pool(x, adj_t_dense)
        x = self.gnn3_embed(x, adj_t)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net_just_balance_pool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()
def train():
    model.train()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in train_loader:
        if data.x.shape[0] == 1 or data.batch[-1] == 0:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y 
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def test(loader):
    model.eval()
    y_true = []
    y_pred = []
    total_loss = 0.
    num_g_total = 0.
    for data in loader:
        if data.x.shape[0] == 1:
            continue
        num_g = data.num_graphs
        num_g_total += num_g
        data = data.to(device)
        x, adj_t = data.x, data.adj_t
        x, adj_t, batch = data.x.to(torch.float32), data.adj_t.to(torch.float32), data.batch
        out = model(x, adj_t, batch)
        pred = out
        is_labeled = data.y == data.y  
        loss = torch.nn.BCEWithLogitsLoss()(pred.to(torch.float32)[is_labeled], data.y.to(torch.float32)[is_labeled])
        total_loss += loss.item() * num_g
        y_true.append(data.y.view(pred.shape)[is_labeled].detach().cpu())
        y_pred.append(torch.sigmoid(pred)[is_labeled].detach().cpu())  
    total_loss = total_loss / num_g_total
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    metric = average_precision_score(y_true, y_pred)
    return total_loss, metric
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 50
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    model = Net_just_balance_pool().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 151):
        loss, _ = train()  
        val_loss, val_acc = test(valid_loader)  
        test_loss, test_acc = test(test_loader)  
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AP: {val_acc:.4f}, Test AP: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val AP: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test AP: {np.std(best_test_accs):.4f}')
print(f'Average Test AP: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.0609, Val AP: 0.1219, Test AP: 0.1238
Seed: 42, Epoch: 002, Loss: 0.0599, Val AP: 0.1301, Test AP: 0.1308
Seed: 42, Epoch: 003, Loss: 0.0593, Val AP: 0.1326, Test AP: 0.1340


KeyboardInterrupt: 